In [7]:
import cv2
import numpy as np

# Load the Haar cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize video capture
cap = cv2.VideoCapture(0)

def calculate_ear(eye):
    # Assuming the eye is detected as a rectangle (ex, ey, ew, eh)
    if len(eye) < 6:
        return 0

    (ex, ey, ew, eh) = eye
    # Calculate eye aspect ratio
    eye_ratio = eh / ew
    return eye_ratio

def estimate_head_pose(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        if len(eyes) == 2:
            eye_coords = [ (ex, ey, ew, eh) for (ex, ey, ew, eh) in eyes]
            eye1, eye2 = eye_coords
            
            ear1 = calculate_ear(eye1)
            ear2 = calculate_ear(eye2)
            
            # Threshold values for EAR can be adjusted
            EAR_THRESHOLD = 0.25
            
            if ear1 < EAR_THRESHOLD or ear2 < EAR_THRESHOLD:
                cv2.putText(frame, "Eyes Closed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(frame, "Eyes Open", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "Eyes Not Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    return frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = estimate_head_pose(frame)
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
